# 使用说明

使用前，请先登录网易云账号，前往需要提取的歌单页面，执行以下步骤

1. 在网页上右键，点击 **审查元素/Inspect Element**
2. 找到 `<tbody>` 标签（应只有一个），右键**复制元素**
3. 在本 Notebook 所在地址 **新建一个名为 [歌单文件名] 的 txt 文件**，将内容复制进去

之后，逐格运行以下代码，即可将歌单提取为 pandas 数据帧并输出为 csv 文件进行存档

In [ ]:
#需要用到bs4
from bs4 import BeautifulSoup

In [ ]:
#导入歌单文件
f = open('[歌单文件名].txt',  'rb')
s = f.readlines()[0]
parsed_html = BeautifulSoup(s)

In [ ]:
# 去除 ‘soil’ 污染
for div in parsed_html.find_all("div", {'class':'soil'}): 
    div.decompose()

In [ ]:
# 分离 tr 元素中的歌曲信息
trows = parsed_html.find_all('tr')

In [ ]:
# 质检，返回值应等于歌单长度
len(trows)

In [ ]:
# 整理歌曲信息
res = []
for r in trows:
    title = r.find('b').text
    duration = r.find("span", {'class':'u-dur candel'}).text
    artist = r.find(lambda tag: 'data-res-author' in tag.attrs)['data-res-author']
    album = r.find(lambda tag: True if "href" in tag.attrs and 'album' in tag["href"] else False).text
    res.append((title, duration, artist, album))

In [ ]:
# 转化为 pandas 数据帧
import pandas as pd
df = pd.DataFrame(res, columns =['标题', '时长', '歌手','专辑']) 
df.head()

In [ ]:
# 可选步骤：输出为 csv
# 如果你的歌单里有 GBK 字符，若用 Excel 直接打开可能出现乱码，记得用 Data > From Text/CSV 规定 UTF-8 格式
df.to_csv('[歌单文件名]_f.txt')